In [2]:
import VGG as vgg
import torch
from HSI_class import HSI
import numpy as np

import os

directory = r"C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment"
files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

directory2 = r"C:\Users\Asus TUF\Documents\code\TA\models\model_800train"
files2 = [os.path.join(directory2, f) for f in os.listdir(directory2) if os.path.isfile(os.path.join(directory2, f))]


models = []

for file in files:
    models.append(file)

for file in files2:
    models.append(file)

for model in models:
    print(model)


C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_134234_0
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_134234_2
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_134234_3
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_134234_5
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_143048_5
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_153154_0
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_153154_1
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_153154_2
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_153154_4
C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_160628_latest
C:\Users\Asus TUF\Documents\code\TA\models\model_800train\model_20250226_222330_0
C:\Users\Asus TUF\Documents\code\

In [3]:
dataset_path = "Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 3:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

Processing file: Hyperspectral oil spill detection datasets\GM01.mat
Processing file: Hyperspectral oil spill detection datasets\GM02.mat
Processing file: Hyperspectral oil spill detection datasets\GM03.mat
Processing file: Hyperspectral oil spill detection datasets\GM04.mat


In [4]:
import zeroPadding
hsi_test = dataset[2]

test_img = hsi_test.img
test_gt = hsi_test.gt

patch_size = 9
half_patch = patch_size // 2

height = test_img.shape[0]
width = test_img.shape[1]

matrix=zeroPadding.zeroPadding_3D(test_img,half_patch) #add 0 in every side of the data
print(f"img shape: {test_img.shape}")
print(f"img shape after padding {matrix.shape}")
print(f"number of pixel {width * height}")

img shape: (1386, 690, 224)
img shape after padding (1394, 698, 224)
number of pixel 956340


In [5]:
print(test_gt.shape)

indices0 = np.argwhere(test_gt == 0)
indices1 = np.argwhere(test_gt == 1)

print(indices0.shape)
print(indices1.shape)

num_samples = 100

random_indices0 = indices0[np.random.choice(len(indices0), num_samples, replace=False)]
random_indices1 = indices1[np.random.choice(len(indices1), num_samples, replace=False)]

test_indices = np.vstack((random_indices0, random_indices1))

print(test_indices.shape)

(1386, 690)
(916980, 2)
(39360, 2)
(200, 2)


In [6]:
def predict(saved_model, input):

    with torch.no_grad():  # Disable gradients for inference
        output = saved_model(input)

    # Convert logits to class label
    predicted_class = torch.argmax(output, dim=1).item()
    confidence = torch.nn.functional.softmax(output, dim=1)[0, predicted_class].item()

    return predicted_class, confidence

In [7]:
scores = []

for model in models:
    print("creating model...")
    saved_model = vgg.VGG16_HSI()
    saved_model.load_state_dict(torch.load(model))
    saved_model.eval()
    print(f"model {model} created")

    total = 0
    correct = 0

    i = 0
    for indice in test_indices:
        x_pos = indice[0]
        y_pos = indice[1]

        true_label = test_gt[x_pos][y_pos]

        selected_rows = matrix[range(x_pos,x_pos+2*half_patch+1), :]
        testing_patch = selected_rows[:, range(y_pos, y_pos+2*half_patch+1)]
        
        # print(i)
        # print(testing_patch[half_patch][half_patch])
        # print(test_img[x_pos][y_pos])

        testing_patch = torch.tensor(testing_patch)
        testing_patch = testing_patch.to(torch.float32)
        testing_patch = testing_patch.unsqueeze(0)
        testing_patch = testing_patch.permute(0, 3, 1, 2)

        prediction, confidence = predict(saved_model, testing_patch)

        print(f"{i+1}: prediction = {prediction}, confidence: {confidence}, expected: {true_label}")
        
        if(prediction == true_label):
            correct += 1

        total += 1
        i += 1

    print(f"skor: {correct}/{total}")
    scores.append((model, correct))


creating model...


c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Asus TUF\AppData\Local\Temp\ipykernel_10128\1627783186.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/p

model C:\Users\Asus TUF\Documents\code\TA\models\model_1000_noaugment\model_20250302_134234_0 created
1: prediction = 0, confidence: 0.8197277188301086, expected: 0
2: prediction = 0, confidence: 0.7226510047912598, expected: 0
3: prediction = 1, confidence: 0.9354747533798218, expected: 0
4: prediction = 1, confidence: 0.8946302533149719, expected: 0
5: prediction = 1, confidence: 0.8300344347953796, expected: 0
6: prediction = 0, confidence: 0.8330591320991516, expected: 0
7: prediction = 1, confidence: 0.9413715600967407, expected: 0
8: prediction = 1, confidence: 0.8705068826675415, expected: 0
9: prediction = 0, confidence: 0.7227067947387695, expected: 0
10: prediction = 1, confidence: 0.864271342754364, expected: 0
11: prediction = 1, confidence: 0.9431484937667847, expected: 0
12: prediction = 1, confidence: 0.6412271857261658, expected: 0
13: prediction = 1, confidence: 0.7224327921867371, expected: 0
14: prediction = 1, confidence: 0.820328414440155, expected: 0
15: predictio

In [8]:
for score in scores:
    print(score)

('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_134234_0', 135)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_134234_2', 163)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_134234_3', 183)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_134234_5', 183)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_143048_5', 183)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_153154_0', 174)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_153154_1', 180)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_153154_2', 180)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_1000_noaugment\\model_20250302_153154_4', 187)
('C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\mod